# Matching catalogs based on membership (detailed)
Here we show the specific steps of matching two catalogs based on proximity

In [ ]:
%load_ext autoreload
%autoreload 2

## ClCatalogs
Given some input data

In [ ]:
import numpy as np
from astropy.table import Table
input1 = Table({'ID': ['CL0a', 'CL1a', 'CL2a', 'CL3a', 'CL4a']})
input1['MASS'] = 1e14*np.arange(1, 6)*10
input2 = Table({'ID': ['CL0b', 'CL1b', 'CL2b', 'CL3b']})
input2['MASS'] = 1e14*np.arange(1, 5)*10
display(input1)
display(input2)
input1_mem = Table(
    {'ID':[
        'MEM0', 'MEM1', 'MEM2', 'MEM3', 'MEM4',
        'MEM5', 'MEM6', 'MEM7', 'MEM8', 'MEM9',
        'MEM10', 'MEM11', 'MEM12', 'MEM13', 'MEM14'],
     'ID_CLUSTER': [
         'CL0a', 'CL0a', 'CL0a', 'CL0a', 'CL0a',
         'CL1a', 'CL1a', 'CL1a', 'CL1a', 'CL2a',
         'CL2a', 'CL2a', 'CL3a', 'CL3a', 'CL4a'],
    })
input2_mem = Table(
    {'ID':[
        'MEM0', 'MEM1', 'MEM2', 'MEM3', 'MEM4',
        'MEM5', 'MEM6', 'MEM7', 'MEM8', 'MEM9',
        'MEM10', 'MEM11', 'MEM12', 'MEM13'],
     'ID_CLUSTER': [
         'CL3b', 'CL0b', 'CL0b', 'CL0b', 'CL0b',
         'CL1b', 'CL1b', 'CL1b', 'CL1b', 'CL2b',
         'CL2b', 'CL2b', 'CL3b', 'CL3b'],
    })
input1_mem['RA'] = np.arange(len(input1_mem))*10.0
input2_mem['RA'] = np.arange(len(input2_mem))*10.0
input1_mem['DEC'] = 0.0
input2_mem['DEC'] = 0.0
input1_mem['Z'] = 0.1
input2_mem['Z'] = 0.1
input1_mem['PMEM'] = 1.0
input2_mem['PMEM'] = 1.0
display(input1_mem)
display(input2_mem)

Create two `ClCatalog` objects. For membership matching it is necessary to have at least `id_cluster` in the member catalogs that corresponds to `id` in the cluster catalogs.
If the matching of the members is done by anngular distance, `ra` and `dec` of the members also must be included.
It is also possible to add the probability of each member belong to the host cluster with a `pmem` argument:

In [ ]:
from clevar.catalog import ClCatalog, MemCatalog
c1 = ClCatalog('Cat1', id=input1['ID'], mass=input1['MASS'])
c2 = ClCatalog('Cat2', id=input2['ID'], mass=input2['MASS'])

m1 = MemCatalog('Mem1', id=input1_mem['ID'], id_cluster=input1_mem['ID_CLUSTER'],
                ra=input1_mem['RA'], dec=input1_mem['DEC'], pmem=input1_mem['PMEM'])
m2 = MemCatalog('Mem2', id=input2_mem['ID'], id_cluster=input2_mem['ID_CLUSTER'],
                ra=input2_mem['RA'], dec=input2_mem['DEC'], pmem=input2_mem['PMEM'])

# Format for nice display
for c in ('mass',):
    c1[c].info.format = '.2e'
    c2[c].info.format = '.2e'

display(c1)
display(c2)
display(m1)
display(m2)

## Matching
Import the `MembershipMatch` and create a object for matching

In [ ]:
from clevar.match import MembershipMatch
mt = MembershipMatch()

### Prepare the matching object
Before matching the clusters it is necessary to match the members catalogs and then filling the clusters with information about of the shared members.

The matching of members can be done by `id` if both member catalogs share the same `id`s or by angular proximity.






The first step is to prepare each catalog with the matching configuration:

- `delta_z`: Defines redshift window for matching. The possible values are:
  - `'cat'`: uses redshift properties of the catalog
  - `'spline.filename'`: interpolates data in `'filename'` assuming (z, zmin, zmax) format
  - `float`: uses `delta_z*(1+z)`
  - `None`: does not use z
- `match_radius`: Radius of the catalog to be used in the matching. If `'cat'` uses the radius in the catalog, else must be in format `'value unit'`. (ex: `'1 arcsec'`, `'1 Mpc'`)

In this case, because one of the configuraion radius has physical units, we also need a cosmology (`cosmo`) object to convert it to angular size (this is done internally).

To match the members by `id`, just run the function:

In [ ]:
%%time
mt.match_members(m1, m2, method='id')

To match the members by angular proximity you also have to provide:
  - `radius`(`str`, `None`). Radius for matching, with format `'value unit'` (ex: `1 arcsec`, `1 Mpc`).
  - `cosmo`(`clevar.Cosmology`, `None`). Cosmology object for when radius has physical units.
Then call the same function with these arguments

In [ ]:
from clevar.cosmology import AstroPyCosmology
mt.match_members(m1, m2, method='angular_distance',
                 radius='0.1 kpc', cosmo=AstroPyCosmology())

This function adds a `matched_mems` attribute to your matching object (`mt.matched_mems` in this case) that contains the indices of the matched members.
This attribute can be saved and loaded so you don't have to redo this step.
Just use the functions:

In [ ]:
mt.save_matched_members(filename='mem_mt.txt', overwrite=False)
mt.load_matched_members(filename='mem_mt.txt')

Now we fill the catalogs with the information regarding the matched members. In this step, each cluster catalog will have a `ClData` table in its `mt_input` attibute with the number of members in each cluster (`nmem`) and a dictionary containing the number of shaded objects with the clusters of the other catalog (`shared_mems`).

If `pmem` is provided to the members, these quantities are computed as:
\begin{equation*}
nmem=\sum_{cluster\;members} Pmem_i
\end{equation*}
\begin{equation*}
shared\_mems=\sum_{shared\;members} Pmem_i
\end{equation*}

In [ ]:
mt.fill_shared_members(c1, c2, m1, m2)

In [ ]:
display(c1.mt_input)
display(c2.mt_input)

Again, these results can be saved and loaded so you don't have to redo this step.
Just use the functions:

In [ ]:
mt.save_shared_members(c1, c2, fileprefix='mem_share')
mt.load_shared_members(c1, c2, fileprefix='mem_share')

Once this step is done, you can actually start matching the clusters.

### Multiple matching
The next step is to match the catalogs and store all candidates that pass the matching criteria.

In [ ]:
%%time
mt.multiple(c1, c2)
mt.multiple(c2, c1)

This will fill the `mt_multi_self` and `mt_multi_other` columns:

In [ ]:
display(c1)
display(c2)

### Unique matching
Once all candidates are stored in each catalog, we can find the best candidates. You can also pass the argument:
- `preference`: In cases where there are multiple matched, how the best candidate will be chosen. Options are: `'more_massive'`, `'angular_proximity'`, `'redshift_proximity'`, `'shared_member_fraction'` (default value).


In [ ]:
%%time
mt.unique(c1, c2, preference='shared_member_fraction')
mt.unique(c2, c1, preference='shared_member_fraction')

This will fill the matching columns:
- `mt_self`: Best candidate found
- `mt_other`: Best candidate found by the other catalog
- `mt_frac_self`: Fraction of shared members with the best candidate found
- `mt_frac_other`: Fraction of shared members by the best candidate found by the other catalog, relative to the other catalog


If `pmem` is present in the members catalogs, the shared fractions are computed by:
\begin{equation*}
\frac{\sum_{shared\;members} Pmem_i}{\sum_{cluster\;members} Pmem_i}
\end{equation*}

In [ ]:
display(c1)
display(c2)

### Cross matching
If you want to make sure the same pair was found in both directions:

In [ ]:
c1.cross_match()
c2.cross_match()

This will fill the `mt_cross` column:

In [ ]:
display(c1)
display(c2)

## Save and Load
The results of the matching can easily be saved and load using `ClEvaR` tools:

In [ ]:
mt.save_matches(c1, c2, out_dir='temp', overwrite=True)

In [ ]:
mt.load_matches(c1, c2, out_dir='temp')
display(c1)
display(c2)